In [1]:
import json
import os
import shutil
from collections import Counter
from multiprocessing.dummy import Pool as ThreadPool

import pandas as pd
import pymysql
import requests

# from predict import predict

In [15]:
"""
db = pymysql.connect(
    host="192.168.173.2",
    port=3306,
    user="root",
    password="root123",
    database="ephoto",
    charset="utf8mb4",
    connect_timeout=31536000,
)
"""

db = pymysql.connect(
    host="192.168.1.86",
    port=33306,
    user="root",
    password="root_mysql8",
    database="ephoto",
    charset="utf8mb4",
    connect_timeout=31536000,
)


def query(sql, commit=False):
    cursor = db.cursor()
    cursor.execute(sql)
    data = cursor.fetchall()
    cursor.close()
    if commit:
        db.commit()
    return data


# db.close()
# db.commit()

In [16]:
df = pd.read_sql(
    "SELECT photo.path,photo.name,photo.code,`photo_album`.name AS album,`tower_type`.name AS type ,`tower_type`.code AS t_code FROM photo  LEFT JOIN `photo_album` ON photo.`album_id`=`photo_album`.id LEFT JOIN `tower` ON `photo_album`.`target_id`=tower.id LEFT JOIN `tower_type` ON tower.`type_code`=`tower_type`.code LEFT JOIN `tower_part_tag` ON photo.id=`tower_part_tag`.`photo_id` WHERE album_id IN (SELECT `album_id`  FROM `task_album` WHERE `status`=2) AND photo.quality=0 AND `tower_part_tag`.`user_id`>1;",
    db,
)

In [17]:
df = df.loc[df["code"].str.len() >= 5]
keys = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13"]
values = [0] * 13
Takeys = [
    "猫头塔",
    "酒杯塔",
    "门型塔",
    "T字塔",
    "干字直线塔",
    "直流同塔并驾直线塔",
    "直流同塔并驾耐张塔",
    "交流同塔并架直线塔",
    "交流同塔并架耐张塔",
    "直流干字耐张塔",
    "交流干字耐张塔",
    "拉V塔",
    "上字塔",
]

for Tower_type in df.groupby("type"):
    Talist = Tower_type[1].values.tolist()
    # count = 0
    Tadict = dict(zip(keys, values))
    for Ta in Talist:
        a = str(Ta[5])
        # print(a)
        # print(Tadict[a])
        Tadict[a] += 1
        # break
    print(Tadict)
    break

{'01': 0, '02': 0, '03': 0, '04': 23185, '05': 0, '06': 0, '07': 0, '08': 0, '09': 0, '10': 0, '11': 0, '12': 0, '13': 0}


In [18]:
df_use = df.loc[df["code"].str.len() >= 5]
df_use

,path,name,code,album,type,t_code
0,SGGA2020/鄂岳Ⅰ线下/#0233-#0298/#0274,IMG_0916.JPG,0100Q,#0274,交流同塔并架直线塔,08
1,SGGA2020/鄂岳Ⅰ线下/#0233-#0298/#0274,IMG_0917.JPG,0200Q,#0274,交流同塔并架直线塔,08
2,SGGA2020/鄂岳Ⅰ线下/#0233-#0298/#0233,IMG_0149.JPG,0100Q,#0233,交流同塔并架耐张塔,09
3,SGGA2020/鄂岳Ⅰ线下/#0233-#0298/#0233,IMG_0150.JPG,0200Q,#0233,交流同塔并架耐张塔,09
4,SGGA2020/鄂岳Ⅰ线下/#0233-#0298/#0233,IMG_0151.JPG,0300Q,#0233,交流同塔并架耐张塔,09
...,...,...,...,...,...,...
165302,SGGA2020/桂北Ⅰ线/#0045-#0001/#0020,IMG_2619.JPG,0700Q,#0020,酒杯塔,02
165303,SGGA2020/桂北Ⅰ线/#0045-#0001/#0020,IMG_2620.JPG,1000Q,#0020,酒杯塔,02
165304,SGGA2020/桂北Ⅰ线/#0045-#0001/#0020,IMG_2622.JPG,1200F,#0020,酒杯塔,02
165305,SGGA2020/桂北Ⅰ线/#0045-#0001/#0020,IMG_2623.JPG,1400Q,#0020,酒杯塔,02


In [19]:
df_line_tower = df.loc[df["t_code"] == "08"]
df_line_tower

,path,name,code,album,type,t_code
2952,SGGA2020/复奉线左侧/#0377-#0414/#0378,IMG_0460.JPG,0100Q,#0378,T字塔,04
2953,SGGA2020/复奉线左侧/#0377-#0414/#0378,IMG_0460.JPG,0100Q,#0378,T字塔,04
2954,SGGA2020/复奉线左侧/#0377-#0414/#0378,IMG_0460.JPG,0100Q,#0378,T字塔,04
2955,SGGA2020/复奉线左侧/#0377-#0414/#0378,IMG_0460.JPG,0100Q,#0378,T字塔,04
2956,SGGA2020/复奉线左侧/#0377-#0414/#0378,IMG_0461.JPG,0200Q,#0378,T字塔,04
...,...,...,...,...,...,...
118629,SGGA2020/鲁固线左侧/#2053-#2017/#2051,IMG_0051.JPG,1000Q,#2051,T字塔,04
118630,SGGA2020/鲁固线左侧/#2053-#2017/#2051,IMG_0053.JPG,1200F,#2051,T字塔,04
118631,SGGA2020/鲁固线左侧/#2053-#2017/#2051,IMG_0053.JPG,1200F,#2051,T字塔,04
118632,SGGA2020/鲁固线左侧/#2053-#2017/#2051,IMG_0054.JPG,1400Q,#2051,T字塔,04


In [23]:
# 文件整理
target_path = "/media/hangpai/libo_training4/"
origin_path = "/media/hangpai/tool/model/"

count = 0
for tower_type in df.groupby("type"):

    count += 1
    print(tower_type[0])
    if count == 2:
        path = target_path + str(tower_type[0])
        if not os.path.exists(path):
            os.makedirs(path)
        Talist = tower_type[1].values.tolist()
        for Ta in Talist:
            # print(Ta[0]+Ta[1])
            code = Ta[2][:2]
            Tapath = os.path.join(path, str(code))
            if not os.path.exists(Tapath):
                os.makedirs(Tapath)
            shutil.copy(
                origin_path + Ta[0] + "/" + Ta[1],
                os.path.join(Tapath, "_".join(Ta[0].split("/")) + "_" + Ta[1]),
            )

            # os.path.join(Tapath, "_".join(Ta[0].split("/")) + "_" + Ta[1]),

T字塔
交流同塔并架直线塔
交流同塔并架耐张塔
交流干字耐张塔
干字直线塔
拉V塔
猫头塔
直流干字耐张塔
直流紧凑耐张塔
酒杯塔
门型塔


In [11]:
# 文件整理
target_path = "/media/hangpai/libo_training2/"
origin_path = "/media/hangpai/tool/model/"


for tower_type in df_use.groupby("type"):
    print(tower_type[0])
    path = os.path.join(target_path, str(tower_type[0]))
    if not os.path.exists(path):
        os.makedirs(path)  # 进入塔型
    Talist = tower_type[1].values.tolist()
    for Ta in Talist:  # 遍历图片
        path_xianlu = os.path.join(
            path, str(Ta[0].split("/")[1]), str(Ta[0].split("/")[2])
        )
        # print(path_xianlu)
        if not os.path.exists(path_xianlu):
            os.makedirs(path_xianlu)  # 进入具体线路
        # print(Ta[0] + Ta[1])
        code = Ta[2][:2]
        Tapath = os.path.join(path_xianlu, str(code))
        if not os.path.exists(Tapath):
            os.makedirs(Tapath)
        shutil.copy(
            os.path.join(origin_path, Ta[0], Ta[1]),
            os.path.join(Tapath, "_".join(Ta[0].split("/")) + "_" + Ta[1]),
        )
        # break
    break

T字塔


In [6]:
for Tower_type in df.groupby("type"):
    print(Tower_type[0])

T字塔
交流同塔并架直线塔
交流同塔并架耐张塔
交流干字耐张塔
干字直线塔
拉V塔
猫头塔
直流干字耐张塔
直流紧凑耐张塔
酒杯塔
门型塔


In [121]:
keys = [
    "01",
    "02",
    "03",
    "0106",
    "0107",
    "0108",
    "0109",
    "0112",
    "0113",
    "0114",
    "0115",
    "0118",
    "0119",
    "0120",
    "0121",
    "0186",
]
# 用的是数字，所以对模型对返回值进行int(str())的操作
values = [
    1,
    2,
    3,
    6,
    7,
    8,
    9,
    [12, 26],
    [12, 26],
    [13, 27],
    [12, 26],
    [10, 24],
    [10, 24],
    [11, 25],
    [14, 28],
    [15, 29],
]
zero_values = [0] * len(values)
partkeys = [
    "塔头",
    "基础",
    "号牌",
    "近地线挂点",
    "近地线防振锤",
    "远地线挂点",
    "远地线防振锤",
    "绝缘子串",
    "绝缘子串左V串",
    "绝缘子串右V串" "绝缘子串左右V串",
    "左V串塔端挂点",
    "塔端挂点",
    "右V串塔端挂点",
    "导线端挂点",
    "导线防振锤",
]
path_origin = "/media/hangpai/tool/model/"
# model_count = 0
# 每个模型不一样，所以大循环只走一遍，最后加 break
for Tower_type in df.groupby("type"):  # 进入具体塔类型
    # model_count += 1
    # if model_count != 1:
    #     continue
    for part_code in Tower_type[1].groupby("code"):  # 根据code进入具体塔部件
        # 位于同一类型塔下面的同一类型部件下
        TaPartdict = dict(zip(keys, values))  # 用于映射
        CandidateDict = dict(zip(keys, zero_values))  # 用于计数
        Part_list = part_code[1].values.tolist()
        count = 0
        totalcount = len(Part_list)
        for part in Part_list:
            time.sleep(0.004)
            code = int(part[2][:2])  # 真值标签
            result = str(predict(path_origin + part[0] + "/" + part[1]))  # 预测值
            # print(part[0]+"/"+part[1])
            # print(code, '   ', result)
            flag = 0
            if code == TaPartdict[result]:
                count += 1
            elif type(code) != type(TaPartdict[result]):
                if code in TaPartdict[result]:
                    count += 1
            elif result in keys:
                CandidateDict[result] += 1
        accuracy = count / totalcount
        print("The accuracy of {} is {:.4f}".format(part_code[0], accuracy))
        with open("record.txt", "a") as f:
            f.write(
                "有{}个{}，模型识别准确率为{:.4f}\n".format(totalcount, part_code[0], accuracy)
            )
        TaPartValues = CandidateDict.values()
        TaPartdict = dict(zip(partkeys, TaPartValues))
        ThreeTa = Counter(TaPartdict).most_common(len(values))
        for i in range(len(values) - 3):
            print(
                "{}的第{}个候选项为:{},数量为：{}\n".format(
                    part_code[0], i + 1, ThreeTa[i][0], ThreeTa[i][1]
                )
            )
            with open("record.txt", "a") as f:
                f.write(
                    "{}的第{}个候选项为:{},数量为：{}\n".format(
                        part_code[0], i + 1, ThreeTa[i][0], ThreeTa[i][1]
                    )
                )

        with open("record.txt", "a") as f:
            f.write("\n\n")
    break

In [97]:
df.loc[df["path"] == "SGGA2020/复奉线左侧/#0689-#0677/#0682"]
# and df["name"] == "IMG_4681.JPG"] ]

# df.loc[df["type"]=='直流同塔并架直线塔']
# 进入具体塔类型的列表：
#     塔code前两位相同的放在同一文件夹，第一次出现则创建文件夹
# for ta in df.groupby("type"):
#     print(ta[1])

,path,name,code,album,type,t_code
20199,SGGA2020/复奉线左侧/#0689-#0677/#0682,IMG_4681.JPG,None,#0682,T字塔,04
20200,SGGA2020/复奉线左侧/#0689-#0677/#0682,IMG_4681.JPG,None,#0682,T字塔,04
20201,SGGA2020/复奉线左侧/#0689-#0677/#0682,IMG_4681.JPG,None,#0682,T字塔,04
20202,SGGA2020/复奉线左侧/#0689-#0677/#0682,IMG_4682.JPG,0200Q,#0682,T字塔,04
20203,SGGA2020/复奉线左侧/#0689-#0677/#0682,IMG_4690.JPG,0600Q,#0682,T字塔,04
20204,SGGA2020/复奉线左侧/#0689-#0677/#0682,IMG_4694.JPG,1000Q,#0682,T字塔,04
20205,SGGA2020/复奉线左侧/#0689-#0677/#0682,IMG_4694.JPG,1000Q,#0682,T字塔,04
20206,SGGA2020/复奉线左侧/#0689-#0677/#0682,IMG_4697.JPG,1200C,#0682,T字塔,04
20207,SGGA2020/复奉线左侧/#0689-#0677/#0682,IMG_4698.JPG,1300C,#0682,T字塔,04
20208,SGGA2020/复奉线左侧/#0689-#0677/#0682,IMG_4699.JPG,1400Q,#0682,T字塔,04


In [123]:
keys = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13"]
values = [0] * 13
Takeys = [
    "T字塔",
    "交流同塔并架直线塔",
    "交流同塔并架耐张塔",
    "交流干字耐张塔",
    "干字直线塔",
    "猫头塔",
    "直流干字耐张塔",
    "酒杯塔",
    "门型塔",
    "上字塔",
    "拉V塔",
    "直流同塔并驾直线塔",
    "交流同塔并驾耐张塔",
]
Tadict = dict(zip(keys, values))
Tadict[str("01")] += 1
Tadict

{'01': 1,
 '02': 0,
 '03': 0,
 '04': 0,
 '05': 0,
 '06': 0,
 '07': 0,
 '08': 0,
 '09': 0,
 '10': 0,
 '11': 0,
 '12': 0,
 '13': 0}

In [7]:
for tower_type in df.groupby("type"):
    print(tower_type[0], len(tower_type[1].groupby("path")))

T字塔 1892
交流同塔并架直线塔 1723
交流同塔并架耐张塔 384
交流干字耐张塔 85
干字直线塔 8
猫头塔 23
直流干字耐张塔 416
酒杯塔 367
门型塔 4


In [9]:
df_missing = df.loc[df["code"].str.len() < 5]

In [8]:
df_missing.to_excel("tag_missing.xls")

<ipython-input-8-03391aae7886>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df_missing.to_excel("tag_missing.xls")


In [9]:
df_useful = df.loc[df["code"].str.len() >= 5]

In [10]:
df.loc[df["type"] == "直流同塔并架直线塔"].to_excel("error.xls")

<ipython-input-10-fbe2f8bb2fda>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.loc[df["type"]=='直流同塔并架直线塔'].to_excel("error.xls")


In [11]:
def de_deup_part(parts):
    final_parts = []
    for part in parts:
        # if len(part) > 5 and part[:5] in final_parts and final_parts[-1] == part[:5]:
        if len(final_parts) > 1 and final_parts[-1] == part[:4]:
            continue
        final_parts.append(part[:4])
    return tuple(final_parts)

In [12]:
tower_type_modes = {}
mode_to_album = {}

for tower_type in df_useful.groupby("type"):
    # print(tower_type[0], len(tower_type[1].groupby("path")))
    tower_type_modes[tower_type[0]] = []
    mode_to_album[tower_type[0]] = {}

    for path in tower_type[1].groupby("path"):
        path[1].sort_values(by="name")
        mode = de_deup_part(list(path[1]["code"].values))
        tower_type_modes[tower_type[0]].append(mode)
        if mode not in mode_to_album[tower_type[0]]:
            mode_to_album[tower_type[0]][mode] = []

        mode_to_album[tower_type[0]][mode].append(path[0])

In [13]:
for t in tower_type_modes:
    items = list(dict(Counter(tower_type_modes[t])).items())
    items = sorted(items, key=lambda x: x[1], reverse=True)
    print("#" * 10)
    print(
        t,
        "塔数量: ",
        len(tower_type_modes[t]),
        "模式数量:",
        len(items),
        "前10种模式累计塔数量:",
        sum([item[1] for item in items[:10]]),
    )
    print("塔数量", "部件顺序模式", "(只取数量大于2的)")
    for item in items:
        if item[1] > 2:
            print(item[1], item[0])

##########
T字塔 塔数量:  1722 模式数量: 420 前10种模式累计塔数量: 659
塔数量 部件顺序模式 (只取数量大于2的)
144 ('0100', '0200', '0900', '0800', '2400', '2600', '2900', '2800', '2700', '2500')
115 ('0200', '0600', '0700', '1000', '1200', '1400', '1500', '1100')
73 ('0100', '0200', '0800', '0900', '2400', '2600', '2800', '2900', '2700', '2500')
73 ('0700', '0600', '1000', '1200', '1500', '1400', '1100')
53 ('0100', '0200', '0600', '0700', '1000', '1200', '1400', '1500', '1300', '1100')
47 ('0100', '0200', '1100', '1300', '1400', '1500', '1200', '1000', '0600', '0700')
46 ('0100', '0600', '1100', '1300', '1200', '1400', '1000')
45 ('0100', '0200', '0900', '0800', '2400', '2600', '2800', '2700', '2500')
36 ('0200', '0600', '0700', '1000', '1200', '1400', '1100')
27 ('0700', '0600', '1000', '1200', '1400', '1100')
26 ('0100', '0600', '0700', '1000', '1200', '1400', '1300', '1100')
25 ('0700', '0700', '0600', '1000', '1200', '1500', '1400', '1100')
24 ('0100', '0200', '0800', '2400', '2600', '2800', '2700', '2500')
23 ('01

In [32]:
with open("mode_sort.txt", "w") as of:
    for t in tower_type_modes:
        items = list(dict(Counter(tower_type_modes[t])).items())
        items = sorted(items, key=lambda x: x[0], reverse=False)
        print("#" * 10, file=of)
        print(
            t,
            "塔数量: ",
            len(tower_type_modes[t]),
            "模式数量:",
            len(items),
            "前10种模式累计塔数量:",
            sum([item[1] for item in items[:10]]),
            file=of,
        )
        print("塔数量", "部件顺序模式", file=of)
        for item in items:
            print(
                item[1], item[0], mode_to_album[t][item[0]], sep="\t", file=of,
            )